# TreePolygons - Baseline DeepForest Model

In this notebook, we illustrate the structure of the datasets and performance of the baseline polygon model against the official training split.

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import torch
import matplotlib

if os.path.basename(os.getcwd()) == 'examples':
    sys.path.append("../")
    
import milliontrees

We will use the DeepForest bounding box model available at https://deepforest.readthedocs.io/en/latest/user_guide/02_prebuilt.html#tree-crown-detection-model. Since a box is a type of four pointed polygon, this can act as the minimum performance for the task.

In [ ]:
from deepforest import main
m = main.deepforest()
m.load_model("weecology/deepforest-tree")

In [ ]:
from deepforest import get_data
from deepforest.visualize import plot_results
from deepforest.utilities import read_file

boxes = m.predict_image(path=get_data("OSBS_029.png"))

print(boxes.head())
plot_results(boxes)

In [ ]:
# Load the box dataset
from milliontrees import get_dataset
from milliontrees.common.data_loaders import get_eval_loader

polygon_dataset = get_dataset("TreePolygons", root_dir="/orange/ewhite/DeepForest/MillionTrees/")
polygon_test_data = polygon_dataset.get_subset("test")

Each iteration of the loader yields metadata, image, targets, let's look at their shapes and structures.

In [ ]:
metadata, image, targets = polygon_test_data[250]

print(f"Metadata shape: {metadata.shape}")
print(f"Image shape: {image.shape}")
print(f"Targets: {targets.keys()}")

In [ ]:
# Mask viz not ready.
#The general workflow is to yield a image, and targets (orange), make a predictions (blue) and evaluate the metric. 
#image_path = polygon_dataset._filename_id_to_code[int(metadata[0])]
#image_path = os.path.join(polygon_dataset._data_dir._str, "images",image_path)

# Load the image, in this case DeepForest expects a numpy array, channels first, 0-255.
#channels_first = image.permute(1, 2, 0).numpy() * 255
#boxes = m.predict_image(channels_first)
#ground_truth = read_file(pd.DataFrame(targets["y"].numpy(),columns=["xmin","ymin","xmax","ymax"]))
#ground_truth["label"] = "Tree"
#plot_results(boxes, ground_truth, image=channels_first.astype("int32"))

## Evaluation example

Every model will output predictions in a slightly different way. MillionTrees expects a dictionary of tensors, same format between predictions and ground truth.

In [ ]:
y_pred = {}
y_pred["y"] = torch.tensor(boxes[["xmin", "ymin", "xmax","ymax"]].values.astype("float32"))
y_pred["labels"] = torch.tensor(boxes.label.apply(
        lambda x: m.label_dict[x]).values.astype(np.int64))
y_pred["scores"] = torch.tensor(boxes.score.values.astype("float32"))

# The eval method takes in batches, so wrap this one example in a liste
# To recover just one metric, you can grab the object directly.
polygon_test_data.eval([y_pred], [targets], metadata=torch.unsqueeze(metadata, dim=0))

The evaluation dictionary is broken down by 'sources' which are individual projects contributed to the MillionTrees project, listed on the 'datasets' page on the docs. For each source the dictionary gives a count of the number of images, and then the bounding box mAP accuracy score. Then it gives the average within-group 'wg' score, and the average over all groups. In this example, we just have 1 image from 1 source, so the rest of groups are nan. Let's look at the entire dataset.

For most evaluation workflows there will be some intermediary code to format the output of whatever prediction workflow to the desired eval format. It would also be nice to have a plotting function to see some examples.

In [15]:
import warnings

def format_deepforest_prediction(images, metadata, targets, m, batch_index):
    # Suppress user warnings to make more readable
    warnings.filterwarnings("ignore")
        
    images = torch.tensor(images)
    predictions = m.predict_step(images, batch_index)
    batch_results = []
    for image_metadata, pred, image_targets, image in zip(metadata,predictions, targets, images):
        basename = polygon_dataset._filename_id_to_code[int(image_metadata[0])]
        if pred is None:
            y_pred = {}
            y_pred["y"] = torch.zeros(4)
            y_pred["labels"] = torch.zeros(1)
            y_pred["scores"] = torch.zeros(1)
        else:
            pred.root_dir = os.path.join(polygon_dataset._data_dir._str, "images")
            pred["image_path"] = basename
            # Reformat to milliontrees format
            y_pred = {}
            y_pred["y"] = torch.tensor(pred[["xmin", "ymin", "xmax","ymax"]].values.astype("float32"))
            y_pred["labels"] = torch.tensor(pred.label)
            y_pred["scores"] = torch.tensor(pred.score.values.astype("float32"))
        batch_results.append(y_pred)
    
    return batch_results, predictions

# Helper function to plot evaluation results
def plot_eval_results(y_pred, pred, image_targets, image, batch_index):  
    basename = pred.image_path.unique()[0]
    ground_truth = read_file(pd.DataFrame(image_targets["y"].numpy(),columns=["xmin","ymin","xmax","ymax"]))
    ground_truth["label"] = "Tree"
    predictions_df = read_file(pred)
    predictions_df["label"] = "Tree"
    # Make image channel last
    image = image.permute(1, 2, 0).numpy()  * 255
    accuracy = polygon_dataset.metrics["recall"]._recall(image_targets["y"], y_pred["y"], iou_threshold=0.3)
    plot_results(predictions_df, ground_truth, image=image.astype("int32"))
    print(f"Image: {basename}, index {batch_index} with Detection Recall: {accuracy.item():.2f}")

In [ ]:
# Get the test loader
test_loader = get_eval_loader("standard", polygon_test_data, batch_size=32)

# Print the length of the test loader
print("There are {} batches in the test loader".format(len(test_loader)))

# Get predictions for the full test set
all_y_pred = []
all_y_true = []

batch_index = 0
for batch in test_loader:
    metadata, images, targets  = batch
    # Get the original DeepForest, and MillionTrees formatted predictions, this is just for plotting, otherwise you just need y_pred.
    milliontrees_format, deepforest_format = format_deepforest_prediction(images, metadata, targets, m, batch_index)
    for image_metadata, y_pred, pred, image_targets, image in zip(metadata, milliontrees_format, deepforest_format, targets, images):
        # Plot every 250th image
        if batch_index % 250 == 0:
            plot_eval_results(y_pred, pred, image_targets, image, batch_index)
        # Gather all predictions and ground truth
        all_y_pred.append(y_pred)
        all_y_true.append(image_targets)
        batch_index += 1

# Evaluate
polygon_dataset.eval(all_y_pred, all_y_true, polygon_test_data.metadata_array)